## Conversational QA Chatbot with Message History

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["USER_AGENT"] = os.getenv("USER_AGENT")

In [2]:
llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")

In [3]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x117e4b790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x117f97a90>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [9]:
import bs4
loader = WebBaseLoader(
    web_paths = ("https://testfully.io/blog/rest-api/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("text-3xl font-semibold text-gray-950 pt-4 pb-6 font-brockmann tracking-tighter", "text-lg text-gray-500", "w-9/12")
        )
    ),
)

docs = loader.load()
docs

[Document(metadata={'source': 'https://testfully.io/blog/rest-api/'}, page_content=' API Development: A 101 Guide to RESTful APIs Explore the comprehensive guide to RESTful APIs, covering definitions, examples, best practices, and FAQs for developers. Dive into the world of modern web development with our in-depth blog post.  RESTful APIs are a key component of web development, allowing for seamless interactions between web applications. They are a development of existing APIs, providing a simpler, more efficient web communication approach. RESTful APIs use HTTP methods (GET, POST, PUT, and DELETE) to perform resource operations, following the Representational State Transfer (REST) principles. This architectural style has become synonymous with modern web services, allowing for scalability, flexibility, and the efficient integration of diverse systems.\nThis blog post explains the definition, importance, and practical applications of RESTful APIs using examples such as Dropbox for file

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [15]:
vector_store = Chroma.from_documents(documents=splits, embedding=embeddings)

In [16]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x19cf8f490>, search_kwargs={})

In [ ]:
##Prompt Template
system_prompt = (
    "Act as an assistant for Question-Answering tasks."
    "Use the retrieved context to answer user's questions."
    "If you don't have enough context to answer, let the user know that you are not sure."
    "Use no more than three sentences while writing an answer."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
]
)

In [19]:
qa_chain = create_stuff_documents_chain(prompt=prompt, llm=llm)
rag_chain = create_retrieval_chain(retriever,qa_chain)

In [23]:
response = rag_chain.invoke({"input": "What is REST API?"})
response

{'input': 'What is REST API?',
 'context': [Document(id='103cfefa-b0ba-4e54-84cd-65bdbc2a108e', metadata={'source': 'https://testfully.io/blog/rest-api/'}, page_content='API Development: A 101 Guide to RESTful APIs Explore the comprehensive guide to RESTful APIs, covering definitions, examples, best practices, and FAQs for developers. Dive into the world of modern web development with our in-depth blog post.  RESTful APIs are a key component of web development, allowing for seamless interactions between web applications. They are a development of existing APIs, providing a simpler, more efficient web communication approach. RESTful APIs use HTTP methods (GET, POST, PUT, and DELETE) to perform resource operations, following the Representational State Transfer (REST) principles. This architectural style has become synonymous with modern web services, allowing for scalability, flexibility, and the efficient integration of diverse systems.'),
  Document(id='73981ba1-772a-491d-a404-f2843e04

In [24]:
response['answer']

'A RESTful API is an application programming interface that follows the Representational State Transfer (REST) principles. It enables interaction with RESTful web services using standard HTTP methods such as GET, POST, PUT, and DELETE.'

### Adding Chat History

- step 1: take user follow-up question and chat history
- step 2: use LLM to rephrase it into a stand alone question
- step 3: run this question through retriever
- step 4: stuff these docs in the context
- step 5: use LLM to get an answer

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# rephrasing chain
context_system_prompt = (
    "You are a helpful assistant. Given the chat history and a follow-up question which might"
    "reference the context in the chat history, rephrase it into a standalone question"
    "which could be understood without the chat history. DO NOT answer the question."
)

context_prompt = ChatPromptTemplate.from_messages([
    ("system", context_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"), ## create_history_aware_retriever is hard-coded to expect {input}
]
)

In [30]:
history_aware_retriever = create_history_aware_retriever(
    llm = llm,
    retriever = retriever,
    prompt = context_prompt
)

In [32]:
##Prompt Template
system_prompt = (
    "Act as an assistant for Question-Answering tasks."
    "Use the retrieved context to answer user's questions."
    "If you don't have enough context to answer, let the user know that you are not sure."
    "Use no more than three sentences while writing an answer."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
]
)

In [34]:
qa_chain = create_stuff_documents_chain(prompt = qa_prompt, llm=llm)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [ ]:
# from langchain_core.messages import AIMessage, HumanMessage
# chat_history = []
# question = "What is API?"
# response = rag_chain.invoke({"input": question, "chat_history":chat_history})

# chat_history.extend(
#     [
#         HumanMessage(content=question),
#         AIMessage(content=response["answer"])
#     ]
# )

In [38]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key= "answer"
)

In [39]:
config = {
    "configurable": {"session_id": "harshini123"}
    }

conversational_rag_chain.invoke(
    {"input": "What is API?"},
    config = config,
)["answer"]


'An API stands for Application Programming Interface, which is a set of defined rules that enables different applications or systems to communicate with each other.'